In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import pandas as pd
import numpy as np
import torch as T
import torch
import math
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import import_ipynb
from model import *
import random
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from pickle import dump,load
#from helper_functions import *
#from dataset import get_data_loader
import torchvision.utils as utils
import argparse
from torch.autograd import Variable
from argparse import ArgumentParser
import matplotlib.pyplot as plt
%matplotlib inline

save_path = "data/saved_models/male/pre/disc/full/model3.tar"
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# if not os.path.exists("data/saved_models"):
#     os.makedirs("data/saved_models")

importing Jupyter notebook from oldModel2.ipynb


In [2]:
ARG_PARSER = ArgumentParser()

ARG_PARSER.add_argument('--num_epochs', default=100, type=int)
ARG_PARSER.add_argument('--seq_len', default=40, type=int)
ARG_PARSER.add_argument('--batch_size', default=100, type=int)
ARG_PARSER.add_argument('--patience', default=30, type=int)
ARG_PARSER.add_argument('--resume_training', default=False)
ARG_PARSER.add_argument('--train', default=True)
ARG_PARSER.add_argument('--evalPred', default=False)
ARG_PARSER.add_argument('--predWin', default=28, type=int)
ARG_PARSER.add_argument('--discriminator', default=True)
ARG_PARSER.add_argument('--attn', default=False)
ARG_PARSER.add_argument('--female', default=False)
ARG_PARSER.add_argument('--male', default=True)

_StoreAction(option_strings=['--clip_value'], dest='clip_value', nargs=None, const=None, default=0.01, type=<class 'float'>, choices=None, help=None, metavar=None)

In [3]:
ARGS = ARG_PARSER.parse_args(args=[])
if ARGS.female:
    INPUT_DIM = 1443
    GENDER = "female"
    G = "F"
if ARGS.male:
    INPUT_DIM = 1457
    GENDER = "male"
    G = "M"
OUTPUT_DIM = 4
AGE_DIM = 40
DRUG_DIM = 380
ENC_EMB_DIM = 256#124
DEC_EMB_DIM = 20#50
ENC_HID_DIM = 512#256
DEC_HID_DIM = 126
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
D_HID_SIZE = 10
MAX_SEQ_LEN = ARGS.seq_len
BATCH_SIZE = ARGS.batch_size
EPSILON = 1e-40


In [4]:
def pred_test(args, model):
    model['g'].eval()
    
    RLoss=0
    TBatches=0
    oBmi=[]
    iBmi=[]
    oAge=[]
    oPids=[]
    oAttn=[]
    oAttnIdx=[]

    with T.autograd.no_grad():
        files = 'data/test/'+GENDER+'/data'+G+'test.csv'
        
        drugFiles = 'data/test/'+GENDER+'/drug/drug'+G+'test.csv'
    
        maskFiles = 'data/test/'+GENDER+'/mask/mask'+G+'test.csv'
        #print(files)http://localhost:8888/notebooks/Desktop/VAE/BRITS/main.ipynb#
        
        dataset = CSVDataset(files, int(args.seq_len*BATCH_SIZE),1356100,args.seq_len,flag=0)
        drugDataset = CSVDataset(drugFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=2)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
        drugLoader = DataLoader(drugDataset,batch_size=1,num_workers=0, shuffle=False)
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        
        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, drugLoader, maskLoader)):
            data,drug,mask=allData
            pids=data[1]
            data=data[0]
            if args.male:
                age=data[:,:,:,1462]
                data=data[:,:,:,1:1458]
            elif args.female:
                age=data[:,:,:,1448]
                data=data[:,:,:,1:1444]
            drug=drug[:,:,:,2:382]
            target=mask[:,:,:,:4]
            mask=mask[:,:,:,4]
            
#             print(data.shape)
#             print(mask.shape)
#             print(target.shape)
            
            age=age.squeeze()
            data=data.squeeze()
            drug=drug.squeeze()
            mask=mask.squeeze()
            target=target.squeeze()
            pids=pids.squeeze()
            pids=pids.view(data.shape[0],data.shape[1])
#             print("pids",pids.shape)
#             print("pids",pids[0])
            testMask = mask.clone().detach()
            y = target.clone().detach()
            #print("ageOutside",age[0])
        
             #------------remove last 5 timestamps------------------
            #print(data[0:10,8:,653])
#             print("=====Initial Data===============")
#             #print("data",data[0,0,:])
#             print("mask",mask[0])
#             print("testMask",testMask[0])
            
            for i in range(data.shape[0]):
                #total length
                j=40
                #prediction window
                k=args.predWin
                #remove values from pred window
                data[i,j-k:j,:]=0
                drug[i,j-k:j,:]=0
                mask[i,j-k:j]=0
                y[i,j-k:j,:]=0
                #zero values in observation window, so that it can be used to check loss in pred win only
                target[i,0:j-k,:]=0
                testMask[i,0:j-k]=0
            
#             print("=====test Data===============")
#             #print("data",data[0,0,:])
#             print("mask",mask[0])
#             print("testMask",testMask[0])
            
#             print("=====idxs Data===============")
            #check where obs win is empty
            emptyIds=torch.sum(data.reshape(data.shape[0],-1),dim=1)
#             print("emptyIds",emptyIds.shape)
#             print("emptyIds",emptyIds)
            idxs = torch.nonzero(emptyIds > 0)
            idxs=idxs.squeeze()
#             print("idxs",idxs.shape)
            #get only data points that have atleast 1 entry in obs window
            data=data[idxs]
            drug=drug[idxs]
            mask=mask[idxs]
            y=y[idxs]
            target=target[idxs]
            testMask=testMask[idxs]
            pids=pids[idxs]
            age=age[idxs]
            
#             print("data",data.shape)
#             print("drug",drug.shape)
#             print("mask",mask.shape)
#             print("target",target.shape)
#             print("pids",pids.shape)
#             print("testMask",testMask.shape)
#             print("age",age.shape)

             #------------E training------------------
            #data=torch.cat((data,y),2)
            data=data.cuda(0)
            drug=drug.cuda(0)
            target=target.cuda(0)
            age=age.cuda(0)
            mask=mask.cuda(0)
            y=y.cuda(0)
            testMask=testMask.cuda(0)
            if args.attn:
                output,attn = model['g'](data, drug, y, age, mask, args.train)
            else:
                output = model['g'](data, drug, y, age, mask, args.train)
            

            #------------R Loss------------------
#             print("======outputs==============")
#             print("output",output.shape)
            
            if args.attn:
                attn = attn.permute(1,0,2)
                attn = attn.reshape(-1,attn.shape[2])
                values, indices = torch.topk(attn,k=10,dim=1)

            output = output.permute(1,0,2)
            #print("output",output.shape)
            output = output.reshape(-1,output.shape[2])
#             print("output",output.shape)
            #print("target",target.shape)
            ##print("output",output[0,0,:])
            ##print("target",target[0,0,:])
            top1 = target.argmax(2)
            top1=top1.view(-1)
#             print("top1",top1.shape)
            

            testMask=testMask.view(-1)
            #print("mask",mask)
            
            #idxs are indices where mask is 1 i:e value is present to calculate supervised loss
            idxs = torch.nonzero(testMask == 1)
#             print("idxs",idxs.shape)
            idxs=idxs.squeeze()
#             print("idxs",idxs.shape)
#             print("idxs",idxs)
            #print("mask",mask[idxs])

            pids=pids.view(-1)
            age=age.view(-1)
            output = output[idxs,:]
            if args.attn:
                values = values[idxs,:]
                indices = indices[idxs,:]
            top1 = top1[idxs]
            age=age[idxs]
            pids=pids[idxs]
#             print("output",output.shape)
#             print("top1",top1.shape)
#             print("age",age.shape)
#             print("pids:",pids[0])
#             print("========================================================")
            #print("pids",pids[0])
            if args.attn:
                values=values.cpu()
                indices=indices.cpu()

            ##print("output",output)
            ##print("top1",top1)
            

            loss = nn.CrossEntropyLoss()
            RLoss = RLoss + loss(output, top1)
            #print("loss",RLoss)

            #Preparing outputs for testing statistics
            oBmi.extend(output)
            iBmi.extend(top1)
            oAge.extend(age)
            oPids.extend(pids)
            if args.attn:
                oAttn.extend(values)
                oAttnIdx.extend(indices)
            #print("oBmi",len(oBmi))
            #print("oBmi",oBmi[0])
            #print("iBmi",len(iBmi))
            #print("oAge",len(oAge))
            #print("oPids",len(oPids))
            
        TBatches=TBatches+batch_idx+1
        RLoss = RLoss/TBatches
    
    #print("===================================")
    #oBmi=np.asarray(oBmi)
    #iBmi=np.asarray(iBmi)
    #oAge=np.asarray(oAge)
    #iBmi=np.asarray(iBmi)
    
    print("RLoss:",RLoss)
    #print("MAE Loss Forward:",mseLossF)
    #print(outputBMI)
    return oBmi,iBmi,oAge,oPids#,oAttn,oAttnIdx
                
                

In [5]:
def run_evalFull(args, model):
    model['g'].eval()
    #model['d'].eval()
    
    RLoss=0
    TBatches=0
    
    with T.autograd.no_grad():
        files = 'data/val/'+GENDER+'/data'+G+'val.csv'
        
        drugFiles = 'data/val/'+GENDER+'/drug/drug'+G+'val.csv'
    
        maskFiles = 'data/val/'+GENDER+'/mask/mask'+G+'val.csv'
        #print(files)
        
        dataset = CSVDataset(files, int(args.seq_len*BATCH_SIZE),1356100,args.seq_len,flag=0)
        drugDataset = CSVDataset(drugFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
        drugLoader = DataLoader(drugDataset,batch_size=1,num_workers=0, shuffle=False)
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        
        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, drugLoader, maskLoader)):
            data,drug,mask=allData
            pids=data[1]
            data=data[0]
            #print(data.shape)
            #print(mask.shape)
            if args.male:
                age=data[:,:,:,1462]#V3Female-1448]#V3Male-1462]#V2-636]#203]
                data=data[:,:,:,1:1458]#V3-Female-1:1444]#V3-Male-1:1458]#V2-1:636]#199]
            elif args.female:
                age=data[:,:,:,1448]#V3Female-1448]#V3Male-1462]#V2-636]#203]
                data=data[:,:,:,1:1444]#V3-Female-1:1444]#V3-Male-1:1458]#V2-1:636]#199]
            drug=drug[:,:,:,2:382]#V2-1:381]
            target=mask[:,:,:,:4]
            mask=mask[:,:,:,4]
            
            
            #print(age.shape)
            #print(data.shape)
            #print(mask.shape)
            #print(target.shape)
            
            age=age.squeeze()
            data=data.squeeze()
            pids=pids.squeeze()
            drug=drug.squeeze()
            mask=mask.squeeze()
            target=target.squeeze()
            
            
            if(args.fine_tune):
                #======================Fine tuning with obs/pred window task=============================
                pids=pids.view(data.shape[0],data.shape[1])
            #             print("pids",pids.shape)
            #             print("pids",pids[0])
                testMask = mask.clone().detach()
                y = target.clone().detach()
                #print("ageOutside",age[0])

                 #------------remove last 5 timestamps------------------
                #print(data[0:10,8:,653])
            #             print("=====Initial Data===============")
            #             #print("data",data[0,0,:])
            #             print("mask",mask[0])
            #             print("testMask",testMask[0])

                for i in range(data.shape[0]):
                    #total length
                    j=40
                    #prediction window
                    k=args.predWin
                    #remove values from pred window
                    data[i,j-k:j,:]=0
                    drug[i,j-k:j,:]=0
                    mask[i,j-k:j]=0
                    y[i,j-k:j,:]=0
                    #zero values in observation window, so that it can be used to check loss in pred win only
                    target[i,0:j-k,:]=0
                    testMask[i,0:j-k]=0

            #             print("=====test Data===============")
            #             #print("data",data[0,0,:])
            #             print("mask",mask[0])
            #             print("testMask",testMask[0])

            #             print("=====idxs Data===============")
                #check where obs win is empty
                emptyIds=torch.sum(data.reshape(data.shape[0],-1),dim=1)
            #             print("emptyIds",emptyIds.shape)
            #             print("emptyIds",emptyIds)
                idxs = torch.nonzero(emptyIds > 0)
                idxs=idxs.squeeze()
            #             print("idxs",idxs.shape)
                #get only data points that have atleast 1 entry in obs window
                data=data[idxs]
                drug=drug[idxs]
                mask=mask[idxs]
                y=y[idxs]
                target=target[idxs]
                testMask=testMask[idxs]
                pids=pids[idxs]
                age=age[idxs]

            #             print("data",data.shape)
            #             print("drug",drug.shape)
            #             print("mask",mask.shape)
            #             print("target",target.shape)
            #             print("pids",pids.shape)
            #             print("testMask",testMask.shape)
            #             print("age",age.shape)

                #data=torch.cat((data,y),2)
                y=y.cuda(0)
                testMask=testMask.cuda(0)
                #========================================================================================
            
            #data=torch.cat((data,target),2)

            #------------E training------------------
            data=data.cuda(0)
            target=target.cuda(0)
            age=age.cuda(0)
            drug=drug.cuda(0)
            mask=mask.cuda(0)
            output = model['g'](data, drug, target, age, mask, args.train)

            #------------R Loss------------------
            #print("output",output.shape)

            output = output.permute(1,0,2)
            #print("output",output.shape)
            output = output.reshape(-1,output.shape[2])

            #print("output",output.shape)
            #print("target",target.shape)
            ##print("output",output[0,0,:])
            ##print("target",target[0,0,:])
            top1 = target.argmax(2)
            
            #print("top1",top1.shape)
            

            mask=mask.view(-1)
            top1=top1.view(-1)
            if(args.fine_tune):
                testMask=testMask.view(-1)
                idxs = torch.nonzero(testMask == 1)
            else:
                idxs = torch.nonzero(mask == 1)
                
            #print("mask",mask.shape)

            
            idxs=idxs.squeeze()
            #print("idxs",idxs.shape)

            output = output[idxs,:]
            top1 = top1[idxs]
            #print("output",output.shape)
            #print("top1",top1.shape)

            ##print("output",output)
            ##print("top1",top1)

            loss = nn.CrossEntropyLoss()
            RLoss = RLoss + loss(output, top1)
            #print("loss",RLoss)

        TBatches=TBatches+batch_idx+1
        #print(TBatches)
    #print(TBatches)
    RLoss = RLoss/TBatches
    #print("===================================")
    print("Val R Loss:",RLoss)
#     print("Val G Loss:",loss['g'])
#     print("Val D Loss:",DLoss/TBatches)
    
    return RLoss
                
                
                


In [6]:
%matplotlib inline
def batch_run(model, data, pids, drug, mask, age, target, args, optimizer, epoch):
    
    rloss=0
    dloss=0
    #print("==============going to encoder=====================")
    loss={}
    freeze_d=False
    #age=age.unsqueeze(2)
    ##print("age",age[0,:,:])
    #print(age.shape)
    #print(data.shape)
    #print(mask.shape)
    #print(target.shape)
    
    if(args.fine_tune):
        #======================Fine tuning with obs/pred window task=============================
        pids=pids.view(data.shape[0],data.shape[1])
    #             print("pids",pids.shape)
    #             print("pids",pids[0])
        testMask = mask.clone().detach()
        y = target.clone().detach()
        #print("ageOutside",age[0])

         #------------remove last 5 timestamps------------------
        #print(data[0:10,8:,653])
    #             print("=====Initial Data===============")
    #             #print("data",data[0,0,:])
    #             print("mask",mask[0])
    #             print("testMask",testMask[0])

        for i in range(data.shape[0]):
            #total length
            j=40
            #prediction window
            k=args.predWin
            #remove values from pred window
            data[i,j-k:j,:]=0
            drug[i,j-k:j,:]=0
            mask[i,j-k:j]=0
            y[i,j-k:j,:]=0
            #zero values in observation window, so that it can be used to check loss in pred win only
            target[i,0:j-k,:]=0
            testMask[i,0:j-k]=0

    #             print("=====test Data===============")
    #             #print("data",data[0,0,:])
    #             print("mask",mask[0])
    #             print("testMask",testMask[0])

    #             print("=====idxs Data===============")
        #check where obs win is empty
        emptyIds=torch.sum(data.reshape(data.shape[0],-1),dim=1)
    #             print("emptyIds",emptyIds.shape)
    #             print("emptyIds",emptyIds)
        idxs = torch.nonzero(emptyIds > 0)
        idxs=idxs.squeeze()
    #             print("idxs",idxs.shape)
        #get only data points that have atleast 1 entry in obs window
        data=data[idxs]
        drug=drug[idxs]
        mask=mask[idxs]
        y=y[idxs]
        target=target[idxs]
        testMask=testMask[idxs]
        pids=pids[idxs]
        age=age[idxs]

    #             print("data",data.shape)
    #             print("drug",drug.shape)
    #             print("mask",mask.shape)
    #             print("target",target.shape)
    #             print("pids",pids.shape)
    #             print("testMask",testMask.shape)
    #             print("age",age.shape)

        #data=torch.cat((data,y),2)
        y=y.cuda(0)
        testMask=testMask.cuda(0)
        #========================================================================================
    
    data=data.cuda(0)
    drug=drug.cuda(0)
    target=target.cuda(0)
    age=age.cuda(0)
    mask=mask.cuda(0)
    
    #------------E training------------------
    optimizer['g'].zero_grad()
    output = model['g'](data, drug, target, age, mask, args.train)
    
    #print("==============Batch Over=============")
    #print("output",output.shape)
        
    output = output.permute(1,0,2)
    #print("output",output.shape)
    #print("output",output[0,0,:])
    out = output.argmax(2)
    #print("out",out.shape)
    #print("out",out[0,:])
    output = output.reshape(-1,output.shape[2])
    #print("output",output.shape)
    #print("target",target.shape)
    ##print("target",target[0,0,:])
    top1 = target.argmax(2)
    #print("top1",top1.shape)
    #print("top1",top1[0,:])
    #print("mask",mask.shape)
    #print("mask",mask[0,:])
    #discriminator
    #------------D training------------------
    if  args.discriminator:
        optimizer['d'].zero_grad()
        mask=mask.float()
        top1=top1.float()
        out=out.float()
        decodedOutput=out*(1-mask)+top1*(mask)
        #print("decodedOutput",decodedOutput.shape)
        #print("decodedOutput",decodedOutput[0,:])
        lossDf = model['d'](decodedOutput, mask, direct="forward")
        lossDb = model['d'](decodedOutput, mask, direct="backward")
        #print("================Outside Discriminator=================")
        dloss=lossDf['loss_d']+lossDb['loss_d']
        if (epoch%2==0):
            dloss.backward(retain_graph=True)
            optimizer['d'].step()
            dloss=dloss.item()
    
    mask=mask.long()
    top1=top1.long()
    top1=top1.view(-1)
    #print("top1",top1.shape)
    
    mask=mask.view(-1)
    
    if(args.fine_tune):
        testMask=testMask.view(-1)
        idxs = torch.nonzero(testMask == 1)
    else:
        idxs = torch.nonzero(mask == 1)
    idxs=idxs.squeeze()
    #print("idxs",idxs.shape)
    #print("i m here")
    output = output[idxs,:]
    top1 = top1[idxs]
    #print("output",output.shape)
    #print("top1",top1.shape)
    
    ##print("output",output)
    ##print("top1",top1)
    
    loss = nn.CrossEntropyLoss()
    rloss = loss(output, top1)
    #print("loss",rloss)
    if  args.discriminator:
        rloss1=rloss+lossDf['loss_g']+lossDb['loss_g']
        rloss1.backward()
    else:
        rloss.backward()
        
    #------------E G training------------------
    #optimizer['e'].zero_grad()
    #optimizer['g'].zero_grad()
    
    #optimizer['e'].step()
    optimizer['g'].step()
    #print("i m here")
    if  args.discriminator:
        return rloss,dloss
    else:
        return rloss

In [7]:
def disc():
    #discriminator
    #------------D training------------------
    if  args.discriminator:
    #         #print("Auto")
        if args.noise:
            #print("Noise")
            #------------Random Noise G training------------------
            h_noise = T.empty([data.shape[0], 400]).uniform_() # random vector
            z_noise=h_noise.repeat(1,args.seq_len).view(h_noise.shape[0],args.seq_len,-1)
            ##print("Noise input",z_noise.shape)
            h_noise=T.unsqueeze(h_noise,0)
            c_noise = T.empty([1, c_n.shape[1], c_n.shape[2]]).uniform_()
            ##print("noise",h_noise.shape,c_noise.shape)
            noiseOutput = model['g'](z_noise,(h_noise,c_noise))
            nLabels = model['d'](noiseOutput.detach(),flag="fake")

            rLabels = model['d'](data,flag="original")
            fLabels = model['d'](decodedOutput.detach(),flag="fake")
            loss['d'] = criterion['d'](rLabels,fLabels,nLabels)

            noise_loss= -T.mean(nLabels)
            #############################################################
            #if No Noise
            optimizer['d'].zero_grad()
            decodedOutput=top1*(mask)+output*(1-mask)
            lossDf = model['d'](decodedOutput, mask, direct="forward")
            lossDb = model['d'](decodedOutput, mask, direct="backward")

            ##print(rLabels.shape,y_real.shape)
            #loss['d'] = F.binary_cross_entropy(rLabels, y_real) + 0.5*(F.binary_cross_entropy(fLabels, y_fake) + F.binary_cross_entropy(nLabels, y_fake))
            loss['d']=lossDf['loss_d']+lossDb['loss_d']
            DLoss=DLoss+loss['d'].item()

            #if not freeze_d:
            ##print(loss['d'])
            #optimizer['d'].zero_grad()
            loss['d'].backward(retain_graph=True)
            optimizer['d'].step()

            loss['gd']= lossDf['loss_g']+lossDb['loss_g']
            GLoss=GLoss+loss['gd'].item()
            loss['g']=loss['g']+loss['gd']


In [8]:
def plotBmi(outBmi, inBmi, age):
    
    outBmi = outBmi.cpu().detach().numpy()
    inBmi = inBmi.cpu().detach().numpy()
    testMask = testMask.cpu().detach().numpy()
    
    
    outBmi=outBmi[np.nonzero(testMask)]
    inBmi=inBmi[np.nonzero(testMask)]
    
    
    return outBmi,inBmi

In [9]:
%matplotlib inline
def run_epoch(args, model, optimizer):
    ''' Run a single epoch
    '''
    
    decodedOutput=[]
    trainLoss=[]
    trainDLoss=[]
    valLoss=[]
    # initialize the early_stopping object
    early_stopping = EarlyStopping(args,patience=args.patience, verbose=True)
    if args.resume_training:
        early_stopping(3.246886, model, optimizer, save_path)
    #for evrey epoch
    for epoch in range(args.num_epochs):
        model['g'].train()
        if  args.discriminator:
            model['d'].train()
        
        ##print(model['e'])
        ##print(model['g'])
    
        #Running Losses
        RLoss=0
        DLoss=0
#         GLoss=0
        TBatches=0  
        #output=[]
        print("=============EPOCH================= ",epoch)
        freeze_d=False
        #if epoch<=args.g_pretraining_epochs:
            #freeze_d=True
       
        path = 'data/train/'+GENDER+'/'
        files = list(map(lambda x : path + x, (filter(lambda x : x.endswith("csv"), os.listdir(path)))))
        
        drugPath = 'data/train/'+GENDER+'/drug/'
        drugFiles = list(map(lambda x : drugPath + x, (filter(lambda x : x.endswith("csv"), os.listdir(drugPath)))))
        
        maskPath = 'data/train/'+GENDER+'/mask/'
        maskFiles = list(map(lambda x : maskPath + x, (filter(lambda x : x.endswith("csv"), os.listdir(maskPath)))))
        
        ids=range(0,len(files))
        for i in ids:
            #print(files[i])
            dataset = CSVDataset(files[i], int(args.seq_len*BATCH_SIZE),1356100,args.seq_len,flag=0)
            drugDataset = CSVDataset(drugFiles[i], int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)
            maskDataset = CSVDataset(maskFiles[i], int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)

            loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
            drugLoader = DataLoader(drugDataset,batch_size=1,num_workers=0, shuffle=False)
            maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

            ##print(files)
            #for every batch
            for batch_idx, allData in enumerate(zip(loader, drugLoader, maskLoader)):
                #bmi_norm=dataset.bmi_norm
                ##print('batch: {}'.format(batch_idx))
                data,drug,mask=allData
                pids=data[1]
                data=data[0]
                #print(data.shape)
                #print(mask.shape)
                #print("drug",drug.shape)
                if args.male:
                    age=data[:,:,:,1462]#V3Female-1448]#V3Male-1462]#V2-636]#203]
                    data=data[:,:,:,1:1458]#V3-Female-1:1444]#V3-Male-1:1458]#V2-1:636]#199]
                elif args.female:
                    age=data[:,:,:,1448]#V3Female-1448]#V3Male-1462]#V2-636]#203]
                    data=data[:,:,:,1:1444]#V3-Female-1:1444]#V3-Male-1:1458]#V2-1:636]#199]
                drug=drug[:,:,:,2:382]#V2-1:381]
                target=mask[:,:,:,:4]
                mask=mask[:,:,:,4]

                #print(age.shape)
                #print(data.shape)
                #print(mask.shape)
                #print(target.shape)

                age=age.squeeze()
                data=data.squeeze()
                pids=pids.squeeze()
                drug=drug.squeeze()
                mask=mask.squeeze()
                target=target.squeeze()
                
               # data=torch.cat((data,target),2)

                #print("age",age[0])
                #print("data",data.shape)
                #print("mask",mask[0,:])
                #print("target",target[0,:,:])

                ##print("age",age[0,:])
                if  args.discriminator:
                    rloss,dloss=batch_run(model, data, pids, drug, mask, age, target, args, optimizer, epoch)
                    #print("Batch Over")
                    RLoss = RLoss + rloss
                    DLoss = DLoss + dloss
                    #print("RLoss",RLoss)
                    #print("DLoss",DLoss)
                else:
                    rloss=batch_run(model, data, pids, drug, mask, age, target, args, optimizer, epoch)
                    RLoss = RLoss + rloss

                T.cuda.empty_cache()
                #paramsE=list(model['e'].parameters())
                #paramsG=list(model['g'].parameters())
                ##print("AFTER PARAM",paramsE[0][20],paramsG[8][0][0])   

            TBatches=TBatches+batch_idx+1
        RLoss=RLoss/TBatches
        DLoss=DLoss/TBatches
#         GLoss=GLoss/TBatches   
        print("Train Loss",RLoss)
        if  args.discriminator:
            print("Disc Loss",DLoss)
            trainDLoss.append(DLoss)
        trainLoss.append(RLoss)

       
        valid_loss = run_evalFull(args, model)
        
        valLoss.append(valid_loss)

        #print("epoch:", epoch, "loss_R:", "%.4f"%RLoss, "loss_G:", "%.4f"%GLoss, "loss_D:", "%.4f"%DLoss)
        #if epoch<1 or epoch >5:
        valid_loss=valid_loss.cpu()
        if not (np.isnan(valid_loss)):
            early_stopping(valid_loss, model, optimizer, save_path)

        if early_stopping.early_stop:
            print("Early stopping")
            break

        #plot_grad_flow(model['e'].named_parameters())
        #plot_grad_flow(model['g'].named_parameters())
        #plot_grad_flow(model['d'].named_parameters())
    return trainLoss, valLoss


In [10]:
def main(args):
    ''' Training sequence
    '''
    train_on_gpu = T.cuda.is_available()
    if train_on_gpu:
        print('Training on GPU.')
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    else:
        print('No GPU available, training on CPU.')
        
    #Create Models
    attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
    enc = Encoder(INPUT_DIM, DRUG_DIM, AGE_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
    dec = Decoder(OUTPUT_DIM, AGE_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

    if  args.discriminator:
        model = {
            'g' : Seq2Seq(enc, dec),
            'd' : Discriminator(MAX_SEQ_LEN, D_HID_SIZE)
        }
    else:
        model = {'g' : Seq2Seq(enc, dec)}

    if  args.discriminator:
        optimizer = {
            'g': optim.Adam(model['g'].parameters(), lr=1e-3),
            'd': optim.Adam(model['d'].parameters(), lr=1e-3)
        }
    else:
        optimizer = {
        'g': optim.Adam(model['g'].parameters(), lr=1e-2)
    }
    
    if torch.cuda.is_available():
        model['g'].cuda(0)
        if  args.discriminator:
            model['d'].cuda(0)
        
    if args.resume_training:
        checkpoint = T.load(save_path)
        model['g'].load_state_dict(checkpoint['G_model'])
        if  args.discriminator:
            model['d'].load_state_dict(checkpoint['D_model'])
        optimizer['g'].load_state_dict(checkpoint['G_trainer'])
        if  args.discriminator:
            optimizer['d'].load_state_dict(checkpoint['D_trainer'])
        #Save Updated Model
        
        trainloss, valLoss = run_epoch(args, model, optimizer) 
        
        return trainloss, valLoss
    
    elif args.train:
        print("================PRE-TRAINING===================")
        trainloss, valLoss = run_epoch(args, model, optimizer) 
        
        return trainloss, valLoss
 
    elif args.evalPred:
        #load Model
        checkpoint = T.load(save_path)
        model['g'].load_state_dict(checkpoint['G_model'])
        if  args.discriminator:
            model['d'].load_state_dict(checkpoint['D_model'])
        optimizer['g'].load_state_dict(checkpoint['G_trainer'])
        if  args.discriminator:
            optimizer['d'].load_state_dict(checkpoint['D_trainer'])
        #oBmi, iBmi, oAge, pids, oAttn,oAttnIdx = pred_test(args, model)
        oBmi, iBmi, oAge, pids = pred_test(args, model)
        
        return oBmi, iBmi, oAge, pids#, oAttn, oAttnIdx
    
    #'epoch': epoch + 1,
    
    
    
        

In [11]:
#oBmi, iBmi, oAge, pids = main(ARGS)
trainloss, valLoss = main(ARGS)

Training on GPU.
================PRE-TRAINING===================
=============EPOCH=================  0
Train Loss tensor(0.9226, device='cuda:0', grad_fn=<DivBackward0>)
Disc Loss 1.3190727031694285
Val R Loss: tensor(0.8144, device='cuda:0')
Validation loss decreased (inf --> 0.814427).  Saving model ...
=============EPOCH=================  1
Train Loss tensor(0.7328, device='cuda:0', grad_fn=<DivBackward0>)
Disc Loss tensor(1.3272, device='cuda:0', grad_fn=<DivBackward0>)
Val R Loss: tensor(0.7008, device='cuda:0')
Validation loss decreased (0.814427 --> 0.700810).  Saving model ...
=============EPOCH=================  2
Train Loss tensor(0.6790, device='cuda:0', grad_fn=<DivBackward0>)
Disc Loss 1.288071586891956
Val R Loss: tensor(0.6771, device='cuda:0')
Validation loss decreased (0.700810 --> 0.677093).  Saving model ...
=============EPOCH=================  3
Train Loss tensor(0.6533, device='cuda:0', grad_fn=<DivBackward0>)
Disc Loss tensor(1.3177, device='cuda:0', grad_fn=<Div

Train Loss tensor(0.5394, device='cuda:0', grad_fn=<DivBackward0>)
Disc Loss 1.2730883094531487
Val R Loss: tensor(0.6408, device='cuda:0')
EarlyStopping counter: 9 out of 30
=============EPOCH=================  33
Train Loss tensor(0.5368, device='cuda:0', grad_fn=<DivBackward0>)
Disc Loss tensor(1.2924, device='cuda:0', grad_fn=<DivBackward0>)
Val R Loss: tensor(0.6447, device='cuda:0')
EarlyStopping counter: 10 out of 30
=============EPOCH=================  34
Train Loss tensor(0.5348, device='cuda:0', grad_fn=<DivBackward0>)
Disc Loss 1.27293061114874
Val R Loss: tensor(0.6432, device='cuda:0')
EarlyStopping counter: 11 out of 30
=============EPOCH=================  35
Train Loss tensor(0.5329, device='cuda:0', grad_fn=<DivBackward0>)
Disc Loss tensor(1.2913, device='cuda:0', grad_fn=<DivBackward0>)
Val R Loss: tensor(0.6444, device='cuda:0')
EarlyStopping counter: 12 out of 30
=============EPOCH=================  36
Train Loss tensor(0.5287, device='cuda:0', grad_fn=<DivBackward0>

In [12]:
import pickle

if ARGS.predWin==12:
    OBS='7'
elif ARGS.predWin==20:
    OBS='5'
elif ARGS.predWin==28:
    OBS='3'

#with open('data/saved_models/'+GENDER+'/pre/disc/full/trainloss3', 'wb') as fp:
#       pickle.dump(trainloss, fp)
    
#with open('data/saved_models/'+GENDER+'/pre/disc/full/valLoss3', 'wb') as fp:
#       pickle.dump(valLoss, fp)

        
#with open('data/saved_models/'+GENDER+'/pre/disc/full/'+OBS+'/oBmi3', 'wb') as fp:
#      pickle.dump(oBmi, fp)
    
#with open('data/saved_models/'+GENDER+'/pre/disc/full/'+OBS+'/iBmi3', 'wb') as fp:
#      pickle.dump(iBmi, fp)

#with open('data/saved_models/'+GENDER+'/pre/disc/full/'+OBS+'/oAge3', 'wb') as fp:
#      pickle.dump(oAge, fp)
#with open('data/saved_models/'+GENDER+'/pre/disc/full/'+OBS+'/pids3', 'wb') as fp:
#      pickle.dump(pids, fp)

# with open('data/saved_models/'+GENDER+'/pre/disc/full/'+OBS+'/attn2/attn', 'wb') as fp:
#     pickle.dump(oAttn, fp)

# with open('data/saved_models/'+GENDER+'/pre/disc/full/'+OBS+'/attn2/oAttnIdx', 'wb') as fp:
#     pickle.dump(oAttnIdx, fp)

In [13]:
# rand_tensor = torch.rand((2, 5, 4, 7))
# rand_tensor

In [14]:
# rand_tensor = rand_tensor.sum(2)
# rand_tensor

In [15]:
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)

In [16]:
# input

In [17]:
# target

In [18]:
# loss = nn.CrossEntropyLoss()
# output = loss(input, target)

In [19]:
# output

In [20]:
# m = nn.Sigmoid()
# loss = nn.BCELoss()
# input = torch.randn(3, requires_grad=True)
# target = torch.empty(3).random_(2)
# output = loss(m(input), target)
# #>>> output.backward()

In [21]:
# m(input)

In [22]:
# target